In [1]:
import cv2 as cv
import os
from os.path import exists
import numpy as np
import mediapipe as mp

objc[33768]: Class CaptureDelegate is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1647765a0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x131fd0860). One of the two will be used. Which one is undefined.
objc[33768]: Class CVWindow is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1647765f0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x111df0a68). One of the two will be used. Which one is undefined.
objc[33768]: Class CVView is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x164776618) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x111df0a90). One of the two will be used. Which one is undefined.
objc[33768]: Class CVSlider is implemented in both /Users/M

## Copy files from orginal dataset

In [8]:
import os
import shutil

# Set the path of the parent directory
parent_directory = "/Users/M374155/Desktop/Preprared New Dataset"

# Create a new directory to copy the files to
new_directory = "/Users/M374155/Desktop/Test/NonSleepy"

# Create the new directory if it doesn't already exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# Counter for the new directory names
counter = 1
for partent_dir_numer in range (1, 20):
    directory = os.path.join(parent_directory, str(partent_dir_numer))
    # Loop through each subdirectory of the parent directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is called "sleepyCombination.avi"
            if file == "nonsleepyCombination.avi":
                # Create the new subdirectory for this file
                new_subdirectory = os.path.join(new_directory, str(counter))
                os.makedirs(new_subdirectory, exist_ok=True)
                # Copy the file to the new subdirectory
                shutil.copy2(os.path.join(root, file), os.path.join(new_subdirectory, file))
                # Increment the counter
                counter += 1


## Create folders for extracted values

In [3]:
DATA_PATH = os.path.join('Extracted_Values_Bigger_Confidence/')

for state in ["Sleepy", "NonSleepy"]:
    for number in range(1,90):
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(number)))
            except:
                pass


### In order to solve openCV problem use version 4.5.5.62

## Extract face features from video

In [3]:
def extractKeypoints(result):
    if result.multi_face_landmarks:
        for face_detected in result.multi_face_landmarks:
            face = np.array([[res.x, res.y, res.z] for res in face_detected.landmark]).flatten()
    else:
        face = np.zeros(1434)

    return face

In [4]:
def drawLandmarks(mp_face_mesh, results, frame): 
    mp_drawing = mp.solutions.drawing_utils  
    mp_drawing_styles = mp.solutions.drawing_styles
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_IRISES, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

In [5]:
def revertColors(frame, mesh): 
    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  
    image.flags.writeable = False 
    results = mesh.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)  
    
    return image, results 

In [7]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

for state in ["Sleepy", "NonSleepy"]:
    for number in range(1,90):
            videoName = "sleepyCombination.avi" if state == "Sleepy" else "nonsleepyCombination.avi"
            videoPath = os.path.join('/Users/M374155/Desktop/Test/', state, str(number), videoName)
            cap = cv.VideoCapture(videoPath)
            frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            print("Processing video number: ", str(number), " with numer of frames: ", frame_count, " for state ", state)
            with mp_face_mesh.FaceMesh(
                refine_landmarks=True,
                min_detection_confidence=0.6,
                min_tracking_confidence = 0.8) as mesh:

                for frame_number in range(frame_count):
                    ret, frame = cap.read()  
                    image, results = revertColors(frame=frame, mesh=mesh)

                    drawLandmarks(mp_face_mesh=mp_face_mesh, results=results, frame=image)

                    landmarks = extractKeypoints(results)
                    npyPath = os.path.join("./Extracted_Values_Bigger_Confidence/", state , str(number), str(frame_number))
                    np.save(npyPath, landmarks)

                    cv.imshow('Video', image)
                    if cv.waitKey(1) == ord('q'):
                        break
            cap.release()
            cv.destroyAllWindows()

Processing video number:  1  with numer of frames:  1502  for state  Sleepy


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processing video number:  2  with numer of frames:  2805  for state  Sleepy


[NULL @ 0x162059d10] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x162059d10] Overread VUI by 8 bits
[h264 @ 0x2933c3cd0] Invalid level prefix
[h264 @ 0x2933c3cd0] error while decoding MB 22 4
[h264 @ 0x2933c3cd0] Invalid level prefix
[h264 @ 0x2933c3cd0] error while decoding MB 6 26
[h264 @ 0x29333d500] illegal short term buffer state detected


Processing video number:  3  with numer of frames:  2748  for state  Sleepy
Processing video number:  4  with numer of frames:  1495  for state  Sleepy
Processing video number:  5  with numer of frames:  2711  for state  Sleepy
Processing video number:  6  with numer of frames:  1351  for state  Sleepy
Processing video number:  7  with numer of frames:  2907  for state  Sleepy
Processing video number:  8  with numer of frames:  2879  for state  Sleepy
Processing video number:  9  with numer of frames:  1380  for state  Sleepy
Processing video number:  10  with numer of frames:  2657  for state  Sleepy
Processing video number:  11  with numer of frames:  3015  for state  Sleepy
Processing video number:  12  with numer of frames:  2922  for state  Sleepy
Processing video number:  13  with numer of frames:  1463  for state  Sleepy
Processing video number:  14  with numer of frames:  3192  for state  Sleepy
Processing video number:  15  with numer of frames:  1424  for state  Sleepy
Proces

...

[NULL @ 0x2a0abc950] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x2a0abc950] Overread VUI by 8 bits
[h264 @ 0x29333f230] Invalid level prefix
[h264 @ 0x29333f230] error while decoding MB 29 4
[h264 @ 0x29333f230] corrupted macroblock 28 25 (total_coeff=-1)
[h264 @ 0x29333f230] error while decoding MB 28 25
[h264 @ 0x2933f2db0] illegal short term buffer state detected
[NULL @ 0x2a0abc950] time_scale/num_units_in_tick invalid or unsupported (0/0)
[h264 @ 0x2933e93d0] corrupted macroblock 29 19 (total_coeff=-1)
[h264 @ 0x2933e93d0] error while decoding MB 29 19
[h264 @ 0x2933e93d0] corrupted macroblock 27 10 (total_coeff=-1)
[h264 @ 0x2933e93d0] error while decoding MB 27 10
[h264 @ 0x29333f230] illegal short term buffer state detected


Processing video number:  80  with numer of frames:  1538  for state  NonSleepy
Processing video number:  81  with numer of frames:  2882  for state  NonSleepy
Processing video number:  82  with numer of frames:  2918  for state  NonSleepy
Processing video number:  83  with numer of frames:  1420  for state  NonSleepy
Processing video number:  84  with numer of frames:  2861  for state  NonSleepy
Processing video number:  85  with numer of frames:  1555  for state  NonSleepy
Processing video number:  86  with numer of frames:  4460  for state  NonSleepy


[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0a3fef0] No start code is found.
[h264 @ 0x2a0a3fef0] Error splitting the input into NAL units.
[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0a25290] No start code is found.
[h264 @ 0x2a0a25290] Error splitting the input into NAL units.
[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0a97390] No start code is found.
[h264 @ 0x2a0a97390] Error splitting the input into NAL units.
[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0af0dc0] No start code is found.
[h264 @ 0x2a0af0dc0] Error splitting the input into NAL units.
[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0ab83f0] No start code is found.
[h264 @ 0x2a0ab83f0] Error splitting the input into NAL units.
[NULL @ 0x2a0a148e0] missing picture in access unit with size 16
[h264 @ 0x2a0a0b7e0] No start code is found.
[h264 @ 0x2a0a0b7e0] Erro

Processing video number:  87  with numer of frames:  3168  for state  NonSleepy
Processing video number:  88  with numer of frames:  1558  for state  NonSleepy
Processing video number:  89  with numer of frames:  2946  for state  NonSleepy


: 